In [1]:
from google.colab import drive #remember to add the features from the data if you can load it correctly
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install SimpleITK


     |████████████████████████████████| 47.4MB 63kB/s 


In [5]:
!pip install pyradiomics

     |████████████████████████████████| 163kB 9.3MB/s 
     |████████████████████████████████| 112kB 9.7MB/s 
     |████████████████████████████████| 552kB 13.3MB/s 


In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=c76a2b5f6745d0787825f95097995a28e295e4f7ba6b95df2ef566be571d2e9a
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=51a8d95d013e35349ec6d59c6a46684a69ca8c604988744fe86e60177ea10584
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
import os
from PIL import Image
from skimage.io import imread
import datetime
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from keras.constraints import unit_norm,min_max_norm,max_norm
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import glob
import radiomics
from radiomics import featureextractor
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
#import sys
#sys.path
#sys.path.append('C:/Users/pensa/Desktop/CAE-for-DM-segmentation/functioncae')
import time
LOG_DIR = f"{int(time.time())}"
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}"

import logging

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [8]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= True,
        fill_mode='reflect')

In [9]:
transform = train_datagen.get_random_transform((124,124)) 
transform



{'brightness': None,
 'channel_shift_intensity': None,
 'flip_horizontal': 0,
 'flip_vertical': 1,
 'shear': -0.11602916531012437,
 'theta': -25.910785289677143,
 'tx': 20.988029782388494,
 'ty': 2.399204803591875,
 'zx': 0.8810918656965091,
 'zy': 1.073963415148892}

In [10]:
import keras
class MassesSequence_radiomics(keras.utils.Sequence):
    """ Classe per il data augmentation per CAE con feature radiomiche """

    def __init__(self, x, y,label_array,features, img_gen, batch_size=10, shape=(124,124)):
        """ Inizializza la sequenza

        Parametri::

        x (np.array): immagini
        y (np.array): maschere
        label_array (np.array): label di classificazione
        features (np.array): feature ottenute con pyradiomics
        batch_size (int): dimensioni della batch
        img_gen (ImageDatagenerator): istanza di ImageDatagenerator
        shape (tuple): shape dell'immagine. Di Default (124, 124)

        """
        self.x, self.y,self.label_array,self.features = x, y,label_array,features
        self.shape = shape
        self.img_gen = img_gen
        self.batch_size = batch_size


    def __len__(self):
        return len(self.x) // self.batch_size

    def on_epoch_end(self):
        """Shuffle the dataset at the end of each epoch."""
        self.x, self.y ,self.label_array,self.features= shuffle(self.x, self.y,
                                                                self.label_array,self.features)

    def process(self, img, transform):
        """ Apply a transformation to an image """
        img = self.img_gen.apply_transform(img, transform)
        return img
            
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_label_array = self.label_array[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_features = self.features[idx * self.batch_size:(idx + 1) * self.batch_size]


        X=[];
        Y=[];
        Classes=[];
        Features=[]
        
        for image, mask,label,feature in zip(self.x, self.y,self.label_array,self.features):
            transform = self.img_gen.get_random_transform(self.shape)
            X.append(self.process(image, transform))
            Y.append(self.process(mask, transform)>0.2)
            Classes.append(label)
            Features.append(feature)

          
        return [np.asarray(X,np.float64),np.asarray(Features,np.float64)], [np.asarray(Y,np.float64) ,np.asarray(Classes,np.float)]

In [11]:
import PIL
from PIL import Image
import cv2
import os
import numpy as np
import glob
from skimage.io import imread
import time
import re
import warnings
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

file_handler = logging.FileHandler('CAE_functions.log')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)


def save_newext(file_name,data_path,ext1,ext2,endpath):
  """Riscrive le immagini in formato leggibile per pyradiomics
  :type file_name: str
  :param file_path: nome del file della immagine

  :type data_path: str
  :param data_path: percorso della cartella dove si trova la immagine

  :type ext1: str
  :param ext1: stringa identificativa dell'estenzione di partenza della immagine

  :type ext2: str
  :param ext2: stringa identificativa dell'estenzione finale della immagine

  :type endpath: str
  :param endpath: percorso della cartella di arrivo
  :returns: dopo aver salvato il nuovo file, restituisce l'esito
  :rtype: bool


  """

  if ext1==ext2:
    logger.debug(f'il file {file_name} in {data_path} ha già la estenzione {ext2}' )
  try:
    image=plt.imread(os.path.join(data_path,file_name))
    file_name=file_name.replace(f'.{ext1}',f'.{ext2}') #insert logging warning if ext1==ext2
    logger.info(f'read {os.path.join(data_path,file_name)} and changed extension from {ext1} to {ext2}')
  except:
    raise Exception('immagine o path non trovati')
    logger.exception(f'Non ho trovato {file_name} in {data_path}')
  status = cv2.imwrite(os.path.join(endpath,file_name),image)
  logger.info(f'ho scritto il file {file_name} in {endpath} come .{ext2} ')
  return status

def unit_masks(file_name,data_path,ext1,ext2, endpath):
  """Normalizza i valori dei pixel delle maschere già nei file per essere utilizzati con pyradiomics. Permette inoltre di cambiare l'estenzione da .pgm a .png o qualunque altra estenzione supportata.

  :type file_name: str
  :param file_path: nome del file della maschera

  :type data_path: str
  :param data_path: percorso della cartella dove si trova la maschera

  :type ext1: str
  :param ext1: stringa identificativa dell'estenzione di partenza della maschera

  :type ext2: str
  :param ext2: stringa identificativa dell'estenzione finale della maschera

  :type endpath: str
  :param endpath: percorso della cartella di arrivo

  :returns: dopo aver salvato il nuovo file, restituisce l'esito e l'immagine
  :rtype: bool, array

  """

  try:
    image=plt.imread(os.path.join(data_path,file_name))
    logger.info(f'Ho letto {file_name} in {data_path}')
  except:
    raise Exception('immagine o path non trovati!')
    logger.exception(f'{file_name} non trovato in {data_path}')
  image=image/255
  file_name=file_name.replace(f'.{ext1}',f'.{ext2}')
  status = cv2.imwrite(os.path.join(endpath,file_name),image)
  logging.info(f'ho scritto {file_name} in {endpath} con successo')
  return status,image

def read_dataset(dataset_path,ext,benign_label,malign_label,x_id ="_resized", y_id="_mass_mask"):
  """Data la cartella con le maschere e le immagini, restituisce i vettori con le immagini, le maschere e le classi. Restituisce i vettori come tensori da dare alla rete.

  :type dataset_path: str
  :param dataset_path: Cartella con le immagini e le relative maschere

  :type data_path: str
  :param data_path: percorso della cartella dove si trova la maschera

  :type ext: str
  :param ext: stringa identificativa dell'estenzione delle immagini e maschere

  :type x_id: str
  :param x_id: identificativo delle immagini

  :type x_id: str
  :param x_id: identificativo delle maschere

  :type benign_label: str
  :param benign_label: identificativo delle masse benigne

  :type malign_label: str
  :param malign_label: identificativo delle masse maligne

  :returns: restituisce i vettori con le immagini, le maschere e le classi
  :rtype: array

  """

  fnames = glob.glob(os.path.join(dataset_path, f"*{x_id}.{ext}"))
  logger.info(f'ho analizzato {dataset_path} cercando le immagini')
  if fnames == []:
    raise Exception('Niente immagini! Il path è sbagliato, magari x_id o ext sono sbagliati! ')
    logger.exception('Non ho trovato immagini in {dataset_path}')
  else:
    pass
  X = []
  Y = []
  class_labels=[]
  for fname in fnames:
      X.append(plt.imread(fname)[1:,1:,np.newaxis])
      Y.append(plt.imread(fname.replace(x_id, y_id))[1:,1:,np.newaxis])
      if benign_label in fname:
        class_labels.append(0)
      elif malign_label in fname:
        class_labels.append(1)
  logger.info(f'ho preso le immagini e le maschere da {dataset_path}, ho trovato tutto e ho creato gli array delle immagini, maschere e classi')
  return np.array(X), np.array(Y) , np.array(class_labels)

def read_dataset_big(dataset_path_mass,dataset_path_mask,benign_label,malign_label,ext='png'):
  """Versione di read_dataset per il dataset del TCIA. Data la cartella con le maschere e le immagini, restituisce i vettori con i filepath delle immagini, le maschere e le classi.

  :type dataset_path_mass: str
  :param dataset_path_mass: Cartella con le immagini

  :type dataset_path_mask: str
  :param dataset_path_mask: percorso della cartella dove si trovano le maschera

  :type ext: str
  :param ext: stringa identificativa dell'estenzione delle immagini e maschere


  :returns: restituisce i vettori con i path delle immagini, le maschere e le classi
  :rtype: array



  """

  fnames  = glob.glob(os.path.join(dataset_path_mass, f"*.{ext}"))
  logger.info(f'ho analizzato {dataset_path_mass} cercando le immagini')

  if fnames == []:
    raise Exception('Niente immagini! Il path è sbagliato, magari ext è sbagliato! ')
    logger.exception('Non ho trovato immagini in {dataset_path_mass}')

  else:
    pass
  masknames = glob.glob(os.path.join(dataset_path_mask, f"*.{ext}"))
  logger.info(f'ho analizzato {dataset_path_mask} cercando le maschere')

  if masknames==[]:
    raise Exception('Immagini o path non trovati!')
    logger.exception('Non ho trovato maschere in {dataset_path_mask}')

  else:
    pass
  X = []
  Y = []
  class_labels=[]
  for fname in fnames:
    try:
      assert(fname.replace(dataset_path_mass, dataset_path_mask) in masknames)
      logger.debug(f'sto verificando che {fname} sia in {dataset_path_mask}')
      Y.append(fname.replace(dataset_path_mass, dataset_path_mask))
      X.append(fname)

      if benign_label in fname:
        class_labels.append(0)
      elif malign_label in fname:
        class_labels.append(1)
    except:
      warnings.warn(f"Attenzione, per {fname} non vi è corrispondenza in {dataset_path_mask}, controlla che l'immagine non sia corrotta o non sia mancante")
      logger.warning(f"Attenzione, per {fname} non vi è corrispondenza in {dataset_path_mask}, controlla che l'immagine non sia corrotta o non sia mancante")
      pass

  return np.array(X), np.array(Y) , np.array(class_labels)

import pickle
import SimpleITK as sitk
import radiomics
from radiomics import featureextractor

def radiomic_dooer(list_test,datapath,endpath,lab,extrc):

  """Funzione per estrarre le feature con pyradiomics e salvarle in un dizionario.

  :type list_test: list
  :param list_test: lista con path immagine e relativa maschera normalizzata
  :type datapath: str
  :param datapath: percorso cartella dove si trova l'immagine
  :type endpath: str
  :param endpath: cartella dove si salva il pickle del dizionario
  :type lab: int
  :param lab: label per indicare la maschera, va da 1 a 255
  :extrc type: str
  :extr param: classe estrattore di pyradiomics

  :returns: dopo aver salvato il pickle, restituisce il tempo impiegato
  :rtype: str


  """

  b=time.perf_counter()
  try:
    logger.debug(f'sto cercando di estrarre le feature da {list[0]} utilizzando come maschera {list[1]}')
    info=extrc.execute(list_test[0],list_test[1],lab)

  except:
    raise Exception('Problema con pyradiomics: forse vi è un problema col label o i path. Controlla che pyradiomics sia installato e che da radiomics sia importato featureextractor')
    logger.exception(f'Problema con pyradiomics: forse vi è un problema col label o i path per {list[0]} e {list[1]} . Controlla che pyradiomics sia installato e che da radiomics sia importato featureextractor')
  c=time.perf_counter()
  logging.info(f'time to extract:{c-b}')
  d=time.perf_counter()
  pattern=re.compile('[M][\w-]*[0-9]*[\w]{13}')
  logging.debug(f'in regex il pattern è {pattern}')
  name=re.findall(pattern,list_test[0])
  logging.debug(f'il patter che sto cercando è {name}')

  dict_r={name[0]:info}
  try:
    with open(os.path.join(endpath,f'feats_{name[0]}.pickle'), 'wb') as handle:
      pickle.dump(dict_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
      logging.info(f'salvato il pickle feats_{name[0]}.pickle in {endpath}')
  except:
    raise Exception('Qualcosa è andato male nel definire il path di arrivo, controlla che endpath sia giusto')
    logger.exception(f'Qualcosa è andato male nel definire il path di arrivo {datapath}, controlla che {endpath} sia un endpath giusto')
  del(dict_r)
  logger.info(f'time to update:{d-c}')

  return 'time to update:{d-c}'

def read_pgm_as_sitk(image_path):
  """Legge un .pgm come una immagine Simple ITK

  :type image_path: str
  :param image_path: path dell'immagine voluta
  :returns: restituisce l'immagine da far leggere a pyradiomics
  :rtype: array
  """

  np_array = np.asarray(PIL.Image.open(image_path))
  logger.info(f'sto leggendo {image_path}')
  sitk_image = sitk.GetImageFromArray(np_array)
  logger.info(f'sto convertendo {image_path}')

  return sitk_image

def dict_update_radiomics(data_path,dictionary):

  """Funzione per unire i vari dizionari creati con radiomic_dooer per poi creare il dataframe

  :type data_path: str
  :param data_path: percorso del pickle da aprire
  :type dictionary: dict
  :param dictionary: dizionario generale del dataframe
  :returns: restituisce il dizionario aggiornato
  :rtype: dict


  """

  with open(data_path, 'rb') as handle:
    logger.info(f'ho aperto {data_path}')
    b = pickle.load(handle)
    logger.info(f'ho caricato {handle}')
    dictionary.update(b)
    logger.info(f'ho fatto un update a {dictionary}')


  return(dictionary)

def blender(img1,img2,a,b):
  """Funzione per sovraimporre due immagini con sfumatura

  :type img1: array numpy
  :param img1: immagine da sovrapporre
  :type img2: array numpy
  :param img2: immagine da svrapporre
  :type a: int or float
  :param a: valore di sfumatura di img1
  :type b: int or float
  :param b: valore di sfumatura di img2
  :returns: restituisce l'immagine sovrapposta
  :rtype: array

  """

  try:
    image=cv2.addWeighted(img1,a, img2, b,0)
    logger.debug('sto cercando di sovrapporre le immagini con pesi rispettivamente {a} e {b}')
  except:
    raise Exception('Sovrapposizione non riuscita. Controllare che le immagini siano giuste e che a e b siano numeri.')
    logger.exception('Sovrapposizione non riuscita. Controllare che le immagini siano giuste e che a e b siano numeri.')
  logger.info('sovrapposizione riuscita')
  return  image

def dice(pred, true, k = 1):
  """Funzione per calcolare l'indice di Dice

  :type pred: array numpy
  :param pred: immagini predette dal modello
  :type true: array numpy
  :param true: immagini target
  :type k: int
  :param k: valore pixel true della maschera
  :returns: restituisce il valore di Dice
  :rtype: float


  """

  intersection = np.sum(pred[true==k]) * 2.0
  try:
    dice = intersection / (np.sum(pred) + np.sum(true))
  except ZeroDivisionError:
    logger.exception('provato a dividere per zero!')
  logger.info(f'calcolato correttamente il dice ottenendo {dice}')
  return dice

def dice_vectorized(pred, true, k = 1):
  """
  Versione vettorizzata per calcolare il coefficiente di dice

  :type pred: array numpy
  :param pred: immagini predette dal modello
  :type true: array numpy
  :param true: immagini target
  :type k: int
  :param k: valore pixel true della maschera
  :returns: restituisce il dice medio
  :rtype: float

  """

  intersection = 2.0 *np.sum(pred * (true==k), axis=(1,2,3))
  try:
    dice = intersection / (pred.sum(axis=(1,2,3)) + true.sum(axis=(1,2,3)))
  except ZeroDivisionError:
    logger.exception('provato a dividere per zero!')
  logger.info(f'calcolato correttamente il dice medio ottenendo {dice}')
  return dice

import matplotlib.pyplot as plt
def modelviewer(model):
  """
  Funzione per visualizzare l'andamento della loss di training e validazione per l'autoencoder e per il classificatore
  :type model: str
  :param model: history del modello di Keras ottenuto dalla funzione

  """

  plt.figure('modelviewer')
  plt.subplot(2,1,1)
  plt.title('autoencoder')
  try:
    plt.plot(model.history['decoder_output_loss'])
    plt.plot(model.history['val_decoder_output_loss'])
  except:
    raise Exception('Attenzione, o model non è un modello Keras o il modello non ha i campi decoder_output_loss o val_decoder_output_loss')
    logger.exception('Attenzione, o model non è un modello Keras o il modello non ha i campi decoder_output_loss o val_decoder_output_loss')
  plt.legend(['loss', 'val_loss'])
  plt.subplot(2,1,2)
  plt.title('classifier')
  try:
    plt.plot(model.history['classification_output_loss'])
    plt.plot(model.history['val_classification_output_loss'])
  except:
    raise Exception('Attenzione, il modello non ha i campi classification_output_loss o val_classification_output_loss')
    logger.exception('Attenzione, il modello non ha i campi classification_output_loss o val_classification_output_loss')
  plt.legend(['loss', 'val_loss'])
  plt.show()


import tensorflow as tf

def heatmap(x,model):
  """
  Funzione che mostra la heatmap dell'ultimo layer convoluzionale prima del classificatore senza funzionalità radiomiche

  :type x: array numpy
  :param x: immagine da segmentare

  :type model: str
  :param model: modello allenato
  :returns: dopo aver plottato la heatmap sovrapposta e l'immagine a cui si riferisce, restituisce la heatmap
  :rtype: array


  """

  img_tensor =x[np.newaxis,...]
  preds = model.predict(img_tensor)[1]
  argmax = np.argmax(preds)
  conv_layer = model.get_layer("last_conv")
  heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])

  with tf.GradientTape() as gtape:
      conv_output, predictions = heatmap_model(img_tensor)
      loss = predictions[1][:, np.argmax(predictions[1])]
      grads = gtape.gradient(loss, conv_output)
      pooled_grads = K.mean(grads, axis=(0, 1, 2))

  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  max_heat = np.max(heatmap)
  if max_heat == 0:
      max_heat = 1e-10
  heatmap /= max_heat


  plt.matshow(heatmap.squeeze())
  plt.show()

  x = np.asarray(255*x, np.uint8)
  heatmap = np.asarray(255*heatmap.squeeze(), np.uint8)


  heatmap = cv2.resize(heatmap, (x.shape[1], x.shape[0]))



  plt.imshow(blender(x,heatmap,1,1))
  plt.axis('off')
  if argmax==1:
    plt.title('the mass is malign')
  else:
    plt.title('the mass is benign')

  return heatmap

import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.keras import models

def heatmap_rad(x,feature,model):
  """
  Funzione che mostra la heatmap dell'ultimo layer convoluzionale prima del classificatore con funzionalità radiomiche

  :type x: array numpy
  :param x: immagine da segmentare
  :type feature: array numpy
  :param feature: feature estratte con pyradiomics
  :type model: class
  :param model: modello allenato
  :returns: dopo aver plottato la heatmap sovrapposta e l'immagine a cui si riferisce, restituisce la heatmap
  :rtype: array
  """

  img_tensor =x[np.newaxis,...]
  feature_tensor=feature[np.newaxis,...]
  preds = model.predict([img_tensor,feature_tensor])[1]
  argmax = np.argmax(preds)
  conv_layer = model.get_layer("last_conv")
  heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])

  with tf.GradientTape() as gtape:
      conv_output, predictions = heatmap_model([img_tensor,feature_tensor])
      loss = predictions[1][:, np.argmax(predictions[1])]
      grads = gtape.gradient(loss, conv_output)
      pooled_grads = K.mean(grads, axis=(0, 1, 2))

  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  max_heat = np.max(heatmap)
  if max_heat == 0:
      max_heat = 1e-10
  heatmap /= max_heat

  plt.figure('heatmap')
  plt.matshow(heatmap.squeeze())
  plt.show()

  x = np.asarray(255*x, np.uint8)
  heatmap = np.asarray(255*heatmap.squeeze(), np.uint8)


  heatmap = cv2.resize(heatmap, (x.shape[1], x.shape[0]))

  plt.figure('Heatactivation')
  plt.imshow(blender(x,heatmap,1,1))
  plt.axis('off')
  if argmax==1:
    plt.title('the mass is malign')
  else:
    plt.title('the mass is benign')
  plt.show()
  return heatmap

def plot_roc_curve(fper, tper,auc):
  """Funzione che fa il plot della curva roc

  :type fper: float
  :param fper: percentuale falsi positivi

  :type tper: float
  :param tper: percentuale veri positivi

  """

  plt.figure('AUC')
  plt.plot(fper, tper, color='orange', label='ROC')
  plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic (ROC) Curve with AUC = %.2f'%auc)
  plt.legend()
  plt.show()

from skimage.filters import threshold_multiotsu

def otsu(image,n_items=2):
  """Funzione che implementa l'algoritmo di Otsu per la segmentazione

  :type image: numpy array
  :param fper: immagine da segmentare
  :type n_items: int
  :param n_items: numero di oggetti da segmentare nell'immagine
  :returns: restituisce l'immagine binarizzata
  :rtype: array

  """

  thresholds = threshold_multiotsu(image,classes=n_items)
  regions = np.digitize(image, bins=thresholds)
  return regions

In [14]:



logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')

file_handler = logging.FileHandler('Tuner.log')
file_handler.setLevel(logging.ERROR)
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stream_handler)

from keras.layers import Conv2D, Conv2DTranspose, Input, Dropout,MaxPooling2D, UpSampling2D, Dense, Flatten
from keras.models import Model, load_model
from keras.layers.experimental.preprocessing import Resizing
from keras.layers.merge import concatenate

def build_model_rad_UNET(hp,shape=(124,124,1),feature_dim=(3,)):
    input_tensor = Input(shape=shape)
    input_vector= Input(shape=feature_dim)

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(input_tensor)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3, 3),activation='relu', kernel_initializer='he_normal',
                            padding='same')(c1)
    p1 =MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Resizing(32,32,interpolation='nearest')(p2)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p2)


    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Resizing(16,16,interpolation='nearest')(p3)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c4)

    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p4)

    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same', name="last_conv")(c5)
#fc layers

    flat=Flatten()(c5)
    flat=concatenate([flat,input_vector])
    den = Dense(hp.Int(f'dense_base_unit',
                                min_value=32,
                                max_value=64,
                                step=4), activation='relu')(flat)

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.


      den = Dense(hp.Int(f'conv_{i}_units',
                                min_value=4,
                                max_value=64,
                                step=4), activation='relu')(den)
      den= Dropout(hp.Float(f'drop_{i}_rate',
                                min_value=0,
                                max_value=.5,
                                step=.1,))(den)

    classification_output = Dense(2, activation = 'sigmoid', name="classification_output")(den)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)

    #c4 = Resizing(14,14,interpolation='nearest')(c4)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    #c3= Resizing(28,28,interpolation='nearest')(c3)

    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Resizing(62,62,interpolation='nearest')(c2)

    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    #c1= Resizing(112,112,interpolation='nearest')(c1)

    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u9)
    c9 = Dropout(0.2)(c9)

    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c9)


    decoder_out = Conv2D(1, (1, 1), activation='sigmoid',name="decoder_output")(c9)

    model = Model([input_tensor,input_vector], [decoder_out,classification_output])
    model.compile(optimizer='adam', loss={'decoder_output':'binary_crossentropy','classification_output':'categorical_crossentropy'},
                  metrics={'decoder_output':'MAE','classification_output':tf.keras.metrics.AUC()})
    return model

datapath='/content/drive/MyDrive/large_sample_Im_segmented_ref'

X_rad,Y_rad,LABELS_rad = read_dataset(datapath,'pgm','_2_resized','_1_resized')
X_rad = X_rad/255
Y_rad = Y_rad/255


"""#In questa sezione estraiamo le feature con pyradiomics e facciamo la PCA

Ora si estraggono le feature e si mettono in un dizionario che verrà poi convertito in un pandas dataframe
"""

x_id ="_resized"
y_id="_mass_mask"
ext='pgm'
fnames = glob.glob(os.path.join(datapath, f"*{x_id}.{ext}"))
fnamesmask = glob.glob(os.path.join(datapath, f"*{y_id}.{ext}"))

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('shape2D')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')

dataframe={f.replace(datapath,''):extractor.execute(read_pgm_as_sitk(f), read_pgm_as_sitk(f.replace(x_id,y_id)),label=255) for f in fnames}


Pandata=pd.DataFrame(dataframe)

Pandataframe=Pandata.drop(Pandata.index[0:22]).T


X_train_rad, X_test_rad, Y_train_rad, Y_test_rad,class_train_rad,class_test_rad,feature_train,feature_test = train_test_split(X_rad, Y_rad,LABELS_rad,Pandataframe, test_size=0.2, random_state=42)



sc = StandardScaler()
feature_train = sc.fit_transform(feature_train)
feature_test = sc.transform(feature_test)


pca = PCA()
feature_train = pca.fit_transform(feature_train)
feature_test = pca.transform(feature_test)
explained_variance = pca.explained_variance_ratio_


pca = PCA(n_components=3)
feature_train = pca.fit_transform(feature_train)
feature_test = pca.transform(feature_test)



train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= True,
        fill_mode='reflect')

transform = train_datagen.get_random_transform((124,124))


X_train_rad_tr, X_train_rad_val, Y_train_rad_tr, Y_train_rad_val,class_train_rad_tr,class_train_rad_val,feature_train_tr,feature_train_val = train_test_split(X_train_rad, Y_train_rad, to_categorical(class_train_rad,2),feature_train, test_size=0.2, random_state=24)

mass_gen_rad = MassesSequence_radiomics(X_train_rad_tr, Y_train_rad_tr,class_train_rad_tr,feature_train_tr, train_datagen)




2021-02-04 09:09:03,338:__main__:ho analizzato /content/drive/MyDrive/large_sample_Im_segmented_ref cercando le immagini
2021-02-04 09:09:03,338:__main__:ho analizzato /content/drive/MyDrive/large_sample_Im_segmented_ref cercando le immagini
2021-02-04 09:09:39,893:__main__:ho preso le immagini e le maschere da /content/drive/MyDrive/large_sample_Im_segmented_ref, ho trovato tutto e ho creato gli array delle immagini, maschere e classi
2021-02-04 09:09:39,893:__main__:ho preso le immagini e le maschere da /content/drive/MyDrive/large_sample_Im_segmented_ref, ho trovato tutto e ho creato gli array delle immagini, maschere e classi
2021-02-04 09:09:39,929:__main__:sto leggendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2_resized.pgm
2021-02-04 09:09:39,929:__main__:sto leggendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2_resized.pgm
2021-02-04 09:09:39,934:__main__:sto convertendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2

In [15]:
tuner = BayesianOptimization(
    build_model_rad_UNET,
    objective= kerastuner.Objective("val_classification_output_auc", direction="max"),
    max_trials=20,
    executions_per_trial=1,
    directory=LOG_DIR)

tuner.search(mass_gen_rad,
             verbose=2,
             epochs=100,
             batch_size=len(mass_gen_rad),
             #callbacks=[tensorboard],
             validation_data=([X_train_rad_val,feature_train_val], [Y_train_rad_val,class_train_rad_val]))

tuner.get_best_hyperparameters()[0].values

newmod=build_modelUNET(tuner.get_best_hyperparameters()[0])
newmod.summary()

tuner.get_best_models()[0].summary()



newmod.save('model_Unet_Tuned') #the model_opt is better



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
dense_base_unit   |36                |?                 
n_layers          |3                 |?                 
conv_0_units      |28                |?                 
drop_0_rate       |0.1               |?                 

Epoch 1/100
11/11 - 13s - loss: 1.2158 - decoder_output_loss: 0.3993 - classification_output_loss: 0.8165 - decoder_output_MAE: 0.2513 - classification_output_auc: 0.5638 - val_loss: 0.9807 - val_decoder_output_loss: 0.2923 - val_classification_output_loss: 0.6884 - val_decoder_output_MAE: 0.1274 - val_classification_output_auc: 0.5838
Epoch 2/100
11/11 - 4s - loss: 0.8886 - decoder_output_loss: 0.2277 - classification_output_loss: 0.6610 - decoder_output_MAE: 0.0917 - classification_output_auc: 0.6107 - val_loss: 0.8430 - val_decoder_output_loss: 0.1745 - val_classification_output_loss: 0.6686 - val_decoder_output_MAE: 0.1015 - val_classification_output_auc: 0.5957
Epoch 3/100


KeyboardInterrupt: ignored